In [ ]:
import exifread
import os
import pandas as pd
import exifread.utils as utils

parent_dir = os.curdir
dest = os.path.join(parent_dir, 'output')
try:
    os.mkdir(dest)
except OSError as error:
    print(error)    

geotags = []

for filename in os.listdir('photos'):
  f = open(os.path.join('photos', filename), 'rb')
  tags = exifread.process_file(f)
  if 'GPS GPSLatitude' in tags:
    lat, lon = utils.get_gps_coords(tags)
    geotags.append([filename, tags['EXIF DateTimeOriginal'], lat, lon])

df = pd.DataFrame(geotags, columns=['filename', 'datetime', 'latitude', 'longitude'])
df.to_csv('output/geodata.tsv', sep="\t")

print(geotags)
display(df)

In [ ]:
import pandas as pd

# Load your data into a DataFrame
data = pd.read_csv('output/geodata.tsv', delimiter="\t", index_col=0)
display(data)

In [ ]:
import folium

# Create a base map
m = folium.Map(location=[data['latitude'].mean(), data['longitude'].mean()], zoom_start=8)

#print(icons)

# Add markers for each accession
for idx, row in data.iterrows():
    acc = row['filename']
    d = row['datetime']
    geodata = [row['latitude'], row['longitude']]
    match acc:
        case '':
            folium.Marker(geodata, icon=folium.Icon(color="red", icon=""), tooltip=acc, popup=f'{d}\n{acc}').add_to(m)
        case _:
            folium.Marker(geodata, icon=folium.Icon(icon=""), tooltip=acc, popup=f'{d}\n{acc}').add_to(m)

# Display the map
m